In [ ]:
library(GenomicRanges)
library(ggplot2)
library(TxDb.Hsapiens.UCSC.hg19.knownGene)
library(ChIPseeker)
library(org.Hs.eg.db)
library(pheatmap)
library(gplots)

## Panel A - ucsc feature annotation

In [ ]:
#eQTL data
all <- readRDS("data/fig3A_all_GReQTLs_GRanges.rds")
female <- readRDS("data/fig3A_female_GReQTLs_GRanges.rds")
male <- readRDS("data/fig3A_male_GReQTLs_GRanges.rds")

# annotate peak
ap.all<- annotatePeak(unique(all), TxDb = TxDb.Hsapiens.UCSC.hg19.knownGene, annoDb = "org.Hs.eg.db")
ap.female<- annotatePeak(unique(female), TxDb = TxDb.Hsapiens.UCSC.hg19.knownGene, annoDb = "org.Hs.eg.db")
ap.male<- annotatePeak(unique(male), TxDb = TxDb.Hsapiens.UCSC.hg19.knownGene, annoDb = "org.Hs.eg.db")

plot_all= ap.all@annoStat
plot_all$Phenotype ="all"
plot_female= ap.female@annoStat
plot_female$Phenotype ="female"
plot_male= ap.male@annoStat
plot_male$Phenotype ="male"

data_for_plot= rbind(plot_all, plot_female, plot_male)
data_for_plot$Feature <- factor(data_for_plot$Feature, levels = c("1st Intron",
                                                                  "Other Intron",
                                                                  "1st Exon",#
                                                                  "Other Exon",
                                                                  "5' UTR",#
                                                                  "3' UTR",
                                                                  "Promoter (<=1kb)",
                                                                  "Promoter (1-2kb)",
                                                                  "Promoter (2-3kb)",
                                                                  "Downstream (<=300)",
                                                                  "Downstream (<=3kb)",
                                                                  "Distal Intergenic"))
data_for_plot= rbind(data_for_plot, c("1st Exon","0", "male noLD")) #this is to add 1st exon to ledgend

cat("2")
#pdf("Annotation_Fig3A.pdf")
ggplot(data_for_plot[ -(grep("noLD",data_for_plot$Phenotype)),], aes(Phenotype, as.numeric(Frequency), fill = Feature)) +
  geom_bar(position = "fill", stat = "identity") + scale_fill_brewer(name='Feature', palette = 'Paired') +
  theme_minimal() + theme(legend.position="bottom") + coord_flip()+ylab("Frequency")+xlab("")
#dev.off()

## Panel B - DeepSea feature annotation

In [ ]:
ds.data <- setNames(data.frame(matrix(ncol = 4, nrow = 0)), c("Feature", "Frequency", "Gender", "Group"))
ds.female <- read.delim("data/fig3B_female_deepSea.evalue",head=T, sep=",")
dim(ds.female)#1218  925
ds.male <- read.delim("data/fig3B_male_deepSea.evalue", head=T, sep=",")
dim(ds.male)#[1] 2199  925
fe_sig=ds.female[,-c(1:6)]
fe_sig[fe_sig > 0.01] = 0
fe_sig[fe_sig!=0] = 1
fe_sig$sum=apply(fe_sig, 1,sum)
fe_sig.percent = (dim(fe_sig[fe_sig$sum>0,])[1]*100)/dim(ds.female)[1]
ds.data[1,] = c("DeepSea features", fe_sig.percent, "females", "DeepSea features")
ds.data[2,] = c("no DeepSea features", 100-fe_sig.percent, "females", "DeepSea features")
m_sig=ds.male[,-c(1:6)]
m_sig[m_sig > 0.01]=0
m_sig[m_sig!=0]=1
m_sig$sum=apply(m_sig, 1,sum)
dim(m_sig[m_sig$sum>0,])#851 = 38.69941%
m_sig.percent = (dim(m_sig[m_sig$sum>0,])[1]*100)/dim(ds.male)[1]
ds.data[3,] = c("DeepSea features", m_sig.percent, "males", "DeepSea features")
ds.data[4,] = c("no DeepSea features", 100-m_sig.percent, "males", "DeepSea features")
# - GRE annotation
gre.female <- readRDS("data/fig3B_female_GReQTLs_GRE.rds")
fe_gre.percent = (length(gre.female@seqnames)*100)/length(female@seqnames)
ds.data[5,] = c("GRE", fe_gre.percent, "females", "GRE")
ds.data[6,] = c("no GRE", 100-fe_gre.percent, "females", "GRE")

gre.male <- readRDS("data/fig3B_male_GReQTLs_GRE.rds")
m_gre.percent = (length(gre.male@seqnames)*100)/length(male@seqnames)
ds.data[7,] = c("GRE", m_gre.percent, "males", "GRE")
ds.data[8,] = c("no GRE", 100-m_gre.percent, "males", "GRE")
# - ARE and ER annotation
are.female <- readRDS("data/fig3B_female_GReQTLs_ARE.rds")
fe_are.percent = (length(are.female@seqnames)*100)/length(female@seqnames)
ds.data[9,] = c("ARE", fe_are.percent, "females", "ARE")
ds.data[10,] = c("no ARE", 100-fe_are.percent, "females", "ARE")
ere.female <- readRDS("data/fig3B_female_GReQTLs_ERE.rds")
fe_ere.percent = (length(ere.female@seqnames)*100)/length(female@seqnames)
ds.data[11,] = c("ERE", fe_ere.percent, "females", "ERE")
ds.data[12,] = c("no ERE", 100-fe_ere.percent, "females", "ERE")

are.male <- readRDS("data/fig3B_male_GReQTLs_ARE.rds")
fe_are.percent = (length(are.male@seqnames)*100)/length(male@seqnames)
ds.data[13,] = c("ARE", fe_are.percent, "males", "ARE")
ds.data[14,] = c("no ARE", 100-fe_are.percent, "males" ,"ARE")
ere.male <- readRDS("data/fig3B_male_GReQTLs_ERE.rds")
fe_ere.percent = (length(ere.male@seqnames)*100)/length(male@seqnames)
ds.data[15,] = c("ERE", fe_ere.percent, "males", "ERE")
ds.data[16,] = c("no ERE", 100-fe_ere.percent, "males", "ERE")

ds.data$Feature <- factor(ds.data$Feature, levels = c("no DeepSea features","DeepSea features","no GRE","GRE","no ARE", "ARE","no ERE","ERE")  )          
#pdf("Annotation_Fig3B.pdf")
ggplot(ds.data, aes(Gender, as.numeric(Frequency), fill = Feature)) +
  geom_bar(position = "fill", stat = "identity") + scale_fill_brewer(name='Feature', palette = 'Paired') +
  theme_minimal() + theme(legend.position="bottom") + coord_flip()+ylab("Frequency")+xlab("")+facet_wrap(~Group)
#dev.off()

## Panel C - ChromHMM annotation

In [ ]:
fe.p <-read.delim("data/fig3C_female_chromHMM_pMatrix.txt", head=T, check.names = F)
fe.or <- read.delim("data/fig3C_female_chromHMM_orMatrix.txt", head=T, check.names = F)
m.p <- read.delim("data/fig3C_male_chromHMM_pMatrix.txt", head=T, check.names = F)
m.or <- read.delim("data/fig3C_male_chromHMM_orMatrix.txt", head=T, check.names = F)
codes <- read.delim("data/fig3C_chromHMM_codes.txt", head=T)

# pheatmap
colnames(fe.or)[-c(1:2)]= paste0("females_",sapply(colnames(fe.or)[-c(1:2)], function(x) unlist(strsplit(x,"_"))[2]))
colnames(m.or)[-c(1:2)]= paste0("males_",sapply(colnames(m.or)[-c(1:2)], function(x) unlist(strsplit(x,"_"))[2]))
colnames(fe.p)[-c(1:2)]= paste0("females_",sapply(colnames(fe.p)[-c(1:2)], function(x) unlist(strsplit(x,"_"))[2]))
colnames(m.p)[-c(1:2)]= paste0("males",sapply(colnames(m.p)[-c(1:2)], function(x) unlist(strsplit(x,"_"))[2]))

both_or=merge(fe.or, m.or, by=c("EID", "Standardized.Epigenome.name"))
both_p=merge(fe.p, m.p, by=c("EID", "Standardized.Epigenome.name"))
both_or[is.na(both_or)]<-0
both_or[both_or=="Inf"]<-0
both_or[both_or=="-Inf"]<-0

#select blood cells only
codes.sub=codes[codes$GROUP %in% c( "Blood & T-cell"),]$EID
both_or_sub=both_or[ both_or$EID %in% codes.sub,]
both_p_sub=both_p[ both_p$EID %in% codes.sub,]
both_or_sort=both_or_sub[ ,sort(colnames(both_or_sub))]
both_p_sort=both_p_sub[ ,sort(colnames(both_p_sub))]

# heatmap
breaks=c(seq(0,8,by=0.02))
color.palette  <- colorRampPalette(c("white", "#2C7BB6"))(length(breaks) - 1)
plot_or=both_or_sort[,-c(32,1)]
plot_p=both_p_sort[,-c(32,1)]
rownames(plot_or)=both_or_sort$Standardized.Epigenome.name
rownames(plot_p)=both_p_sort$Standardized.Epigenome.name

#plot only sig. p-values
plot_p[plot_p>0.05] = ""
#pdf("Annotation_Fig3C.pdf")
heatmap.2((as.matrix(plot_or)), cellnote=(as.matrix(plot_p)),
          notecol="black",col=color.palette,scale="none",key=TRUE, keysize=1, Rowv = F,Colv = F,dendrogram = "none",
          density.info="none", trace="none", cexRow=0.5,cexCol=0.5, notecex =0.2)
#dev.off()